In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [4]:
df = pd.read_csv('/content/drive/MyDrive/water_portability.csv') #You might have to upload the dataset again or change this directory


In [12]:
#Filling null values
df['ph'].fillna(df['ph'].mean(), inplace=True)
df['Sulfate'].fillna(df['Sulfate'].mean(), inplace=True)
df['Trihalomethanes'].fillna(df['Trihalomethanes'].mean(), inplace=True)


df_train = df.iloc[:2500, :] # splitting data into training and testing set
df_test = df.iloc[2500:, :9]
df_test2 = df_train.iloc[100:800, :]
df_test_labels = df.iloc[2500:, 9]
df_test2_labels = df_train. iloc[100:800,9]# testing labels to compare with later

In [8]:
df_p = df_train[df['Potability']==1] #Splitting data frame into Potable and non_potable for easy comparison
df_np = df_train[df['Potability']==0]

<ipython-input-8-6db0a1153228>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_p = df_train[df['Potability']==1] #Splitting data frame into Potable and non_potable for easy comparison
<ipython-input-8-6db0a1153228>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_np = df_train[df['Potability']==0]


In [10]:
#plotting the pdf functions for visualisation (not used here, but useful for bugs)
def plot_normal_dist(data):
    mean = data.mean()
    std = data.std()
    x = np.linspace(data.min(), data.max(), data.count())

    plt.plot(x, norm.pdf(x, mean, std))

In [9]:
def given_prob(given_value, data_column):
          mean = data_column.mean()
          std = data_column.std()
          given_normal_prob = norm.pdf(given_value, mean, std) #basically give the y values in a gauss distribution for x values
          return given_normal_prob

def is_potable(values_list):
        #Gauss naive bayes probability func. for potable and non-potable by multiplying probability of potable from given data and prob. of each factor given potable or non-potable

        values_list = np.array([ph, hardness, solids, chloram, sulf, condu, org_c, tri_h_meth, turb]) #list of variable names so it's easier to loop the process

        column_names = list(df.columns)
        column_names.pop()

        def multipy_all_prob(data):
            prob = data['ph'].count()/df['ph'].count() #inital Probability P(potable or np) via column length of potable/total, I took ph column length, they're all the same length so it doesn't matter
            for i in range(0, 9):
              given_probs = given_prob(values_list[i], data[column_names[i]])
              prob *= given_probs
            return prob

        #Final posterior probability for potable and non-potable dataframes
        post_prob_npotable = np.log(multipy_all_prob(df_np)) #we take log so it converts really small decimals into numbers readable by the computer
        post_prob_potable = np.log(multipy_all_prob(df_p))


        if post_prob_potable > post_prob_npotable:
          is_potable = 1
        else:
          is_potable = 0

        return is_potable

Final Code

In [11]:
#You can play around with the values here, it was too tedious to make it into an input interface

ph = 7.080795              #pH
hardness = 204.890455      #Hardness
solids = 20791.318981    #Solids
chloram = 7.300212         #Chloramines
sulf = 368.516441          #Sulfate
condu = 564.308654         #Conductivity
org_c = 10.379783         #Organic_c
tri_h_meth = 86.990970     #Trihalomethanes
turb = 2.963135           #Turbidity

values_list = np.array([ph, hardness, solids, chloram, sulf, condu, org_c, tri_h_meth, turb])

is_drinkable = is_potable(values_list)

if is_drinkable==1:
    print("The water is safe for drinking!")
else:
    print("Don't drink it.")

Don't drink it.


Testing Accuracy

In [13]:
prob_list = []
row_values_list = []

for index, row in df_test2.iterrows(): #running the is_potable function through each row in df_test and putting results in a list
    row_values = np.array(row)
    prob_list.append(is_potable(row_values))
    # prob_list.append(is_potable(row_values))

In [14]:
prob_array = np.array(prob_list)
test_labels = df_test2_labels.to_numpy()

print(len(prob_array))
print(len(test_labels))

700
700


In [15]:
#Calculating

matches = np.sum(prob_array == test_labels)

accuracy = matches/len(test_labels)

print(f"Accuracy is : {accuracy*100}%")

Accuracy is : 57.14285714285714%
